<a href="https://colab.research.google.com/github/gshivani4/E-Commerce-Funnel-Intelligence-Mobile-to-Desktop-Gap-Analysis/blob/main/E_Commerce_Funnel_Intelligence_Mobile_to_Desktop_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving event_data.csv to event_data.csv


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)


In [6]:
# If you upload the file directly in Colab each time:
df = pd.read_csv("/content/event_data.csv")

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   event_id           50000 non-null  int64  
 1   event_ts           49078 non-null  object 
 2   user_id            46506 non-null  object 
 3   session_id         50000 non-null  object 
 4   event_type         50000 non-null  object 
 5   product_id         37563 non-null  object 
 6   content_id         34976 non-null  object 
 7   search_keyword     2139 non-null   object 
 8   price              3762 non-null   object 
 9   platform           50000 non-null  object 
 10  country            50000 non-null  object 
 11  city               50000 non-null  object 
 12  device_type        50000 non-null  object 
 13  browser            42786 non-null  object 
 14  network_type       40031 non-null  object 
 15  os_version         50000 non-null  object 
 16  session_duration   248

In [4]:
df.columns

Index(['event_id', 'event_ts', 'user_id', 'session_id', 'event_type',
       'product_id', 'content_id', 'search_keyword', 'price', 'platform',
       'country', 'city', 'device_type', 'browser', 'network_type',
       'os_version', 'session_duration', 'page_scroll_depth',
       'clicks_in_session', 'ingest_ts', 'raw_json'],
      dtype='object')

In [9]:
# 🔁 Map your real column names to generic names used in the code

COL_EVENT_TIME  = "event_ts"
COL_EVENT_NAME  = "event_type"
COL_USER_ID     = "user_id"
COL_SESSION_ID  = "session_id"      # you already have this
COL_DEVICE      = "device_type"
COL_PLATFORM    = "platform"
COL_COUNTRY     = "country"
COL_CITY        = "city"
COL_PRODUCT_ID  = "product_id"
COL_SEARCH_KW   = "search_keyword"
COL_REVENUE     = "price"
COL_NETWORK     = "network_type"
COL_BROWSER     = "browser"
COL_OS          = "os_version"


In [10]:
df = df.copy()

# 1) Drop exact duplicate rows
df = df.drop_duplicates()

# 2) Parse timestamps
df[COL_EVENT_TIME] = pd.to_datetime(df[COL_EVENT_TIME], errors="coerce")
df = df.dropna(subset=[COL_EVENT_TIME])

# 3) Sort by user + time
df = df.sort_values(by=[COL_USER_ID, COL_EVENT_TIME])

# 4) Standardize event names
df[COL_EVENT_NAME] = df[COL_EVENT_NAME].str.lower().str.strip()


/tmp/ipython-input-154663608.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[COL_EVENT_TIME] = pd.to_datetime(df[COL_EVENT_TIME], errors="coerce")


In [11]:
SESSION_GAP_MINUTES = 30

df["prev_time"] = df.groupby(COL_USER_ID)[COL_EVENT_TIME].shift(1)
df["time_diff_min"] = (df[COL_EVENT_TIME] - df["prev_time"]).dt.total_seconds() / 60

df["new_session_flag"] = (df["time_diff_min"].isna()) | (df["time_diff_min"] > SESSION_GAP_MINUTES)

df["session_index"] = df.groupby(COL_USER_ID)["new_session_flag"].cumsum()

df["session_id"] = df[COL_USER_ID].astype(str) + "_" + df["session_index"].astype(str)

df = df.drop(columns=["prev_time", "new_session_flag"])

In [12]:
df["event_date"] = df[COL_EVENT_TIME].dt.date
df["event_hour"] = df[COL_EVENT_TIME].dt.hour
df["event_dayofweek"] = df[COL_EVENT_TIME].dt.day_name()

# Event order within session
df["event_rank_in_session"] = df.groupby("session_id")[COL_EVENT_TIME].rank(method="first").astype(int)

# Time since previous event in session
df["prev_time_in_session"] = df.groupby("session_id")[COL_EVENT_TIME].shift(1)
df["time_since_prev_event_sec"] = (
    df[COL_EVENT_TIME] - df["prev_time_in_session"]
).dt.total_seconds()
df["time_since_prev_event_sec"] = df["time_since_prev_event_sec"].fillna(0)
df = df.drop(columns=["prev_time_in_session"])

In [13]:
session_agg = df.groupby("session_id").agg(
    user_id=(COL_USER_ID, "first"),
    session_start=(COL_EVENT_TIME, "min"),
    session_end=(COL_EVENT_TIME, "max"),
    event_count=(COL_EVENT_NAME, "count"),
    unique_events=(COL_EVENT_NAME, pd.Series.nunique),
)

session_agg["session_duration_sec"] = (
    session_agg["session_end"] - session_agg["session_start"]
).dt.total_seconds()

# Define bounce: only one event in session
session_agg["is_bounce"] = (session_agg["event_count"] == 1).astype(int)

session_agg.head()


,user_id,session_start,session_end,event_count,unique_events,session_duration_sec,is_bounce
session_id,,,,,,,
nan_nan,None,2025-11-25 14:18:49.369890,2025-11-30 14:18:00.369890,3124,20,431951.0,0
user_1000_1.0,user_1000,2025-11-25 18:57:29.369890,2025-11-25 18:57:29.369890,1,1,0.0,1
user_1000_2.0,user_1000,2025-11-27 00:06:17.369890,2025-11-27 00:06:17.369890,1,1,0.0,1
user_1001_1.0,user_1001,2025-11-28 00:57:16.369890,2025-11-28 00:57:16.369890,1,1,0.0,1
user_1001_2.0,user_1001,2025-11-28 16:33:21.369890,2025-11-28 16:33:21.369890,1,1,0.0,1


In [14]:
# Example if you have scroll_depth column
if "scroll_depth" in df.columns:
    scroll_agg = df.groupby("session_id")["scroll_depth"].max().rename("max_scroll_depth")
    session_agg = session_agg.join(scroll_agg)
else:
    session_agg["max_scroll_depth"] = np.nan


In [15]:
# Define "engagement_actions" as non-trivial events
engagement_events = ["product_click", "add_to_cart", "purchase", "signup", "checkout"]
df["is_engagement_event"] = df[COL_EVENT_NAME].isin(engagement_events).astype(int)

engagement_per_session = df.groupby("session_id")["is_engagement_event"].sum().rename("engagement_actions")

session_agg = session_agg.join(engagement_per_session)

# Avoid divide by zero
session_agg["session_duration_sec"] = session_agg["session_duration_sec"].replace(0, 1)

session_agg["session_quality_score"] = (
    (session_agg["engagement_actions"] * 2 - session_agg["is_bounce"] * 3)
    / np.log1p(session_agg["session_duration_sec"])
)

session_agg["session_quality_score"].describe()


,session_quality_score
count,41212.000000
mean,-4.104116
std,0.854918
min,-4.328085
25%,-4.328085
50%,-4.328085
75%,-4.328085
max,19.420364


In [16]:
df.to_csv("clean_events.csv", index=False)
session_agg.to_csv("sessions.csv", index=True)


In [18]:
df = pd.read_csv("clean_events.csv", parse_dates=["event_ts"])

In [19]:
FUNNEL_STEPS = [
    "page_view",
    "search",
    "product_click",
    "add_to_cart",
    "purchase",
]

# Keep only events related to these steps
funnel_df = df[df[COL_EVENT_NAME].isin(FUNNEL_STEPS)].copy()


In [21]:
user_funnel_times = (
    funnel_df
    .groupby([COL_USER_ID, COL_EVENT_NAME])["event_ts"]
    .min()
    .unstack(COL_EVENT_NAME)
)

user_funnel_times.head()


event_type,add_to_cart,page_view,search
user_id,,,
user_1,2025-11-25 21:02:10.369890,NaT,NaT
user_10,NaT,2025-11-25 19:46:55.369890,NaT
user_1002,NaT,NaT,2025-11-29 08:43:12.369890
user_1004,2025-11-25 15:15:50.369890,NaT,NaT
user_1007,NaT,NaT,2025-11-26 06:05:50.369890


In [22]:
def user_reached_step(row, step_idx):
    """Check if user reached FUNNEL_STEPS[step_idx] in correct order."""
    prev_times = []
    for i in range(step_idx + 1):
        step = FUNNEL_STEPS[i]
        t = row.get(step)
        if pd.isna(t):
            return False
        prev_times.append(t)
    # strictly non-decreasing times
    return prev_times == sorted(prev_times)

results = []

for i, step in enumerate(FUNNEL_STEPS):
    reached = user_funnel_times.apply(lambda r: user_reached_step(r, i), axis=1)
    results.append(
        {
            "step": step,
            "users_reached": reached.sum()
        }
    )

funnel_counts = pd.DataFrame(results)
funnel_counts["conversion_rate_from_start"] = (
    funnel_counts["users_reached"] / funnel_counts["users_reached"].iloc[0]
)

funnel_counts


,step,users_reached,conversion_rate_from_start
0,page_view,1847,1.000000
1,search,196,0.106118
2,product_click,0,0.000000
3,add_to_cart,0,0.000000
4,purchase,0,0.000000


In [23]:
# Merge device info
user_device = df.groupby(COL_USER_ID)[COL_DEVICE].agg(lambda x: x.value_counts().index[0])
uf = user_funnel_times.join(user_device)

def compute_funnel_by_group(group_col):
    out = []
    for group_value, sub in uf.groupby(group_col):
        row = {"group": group_value}
        base = len(sub)
        for i, step in enumerate(FUNNEL_STEPS):
            reached = sub.apply(lambda r: user_reached_step(r, i), axis=1).sum()
            row[f"{step}_count"] = reached
            row[f"{step}_rate_from_start"] = reached / base if base else 0
        out.append(row)
    return pd.DataFrame(out)

funnel_by_device = compute_funnel_by_group(COL_DEVICE)
funnel_by_device


,group,page_view_count,page_view_rate_from_start,search_count,search_rate_from_start,product_click_count,product_click_rate_from_start,add_to_cart_count,add_to_cart_rate_from_start,purchase_count,purchase_rate_from_start
0,flagship,352,0.400000,50,0.056818,0,0.0,0,0.0,0,0.0
1,high_end,355,0.412791,37,0.043023,0,0.0,0,0.0,0,0.0
2,low_end,401,0.433514,23,0.024865,0,0.0,0,0.0,0,0.0
3,mid_range,384,0.434881,39,0.044168,0,0.0,0,0.0,0,0.0
4,unknown,355,0.404789,47,0.053592,0,0.0,0,0.0,0,0.0


In [25]:
df = pd.read_csv("clean_events.csv", parse_dates=["event_ts"])

# First activity date per user
user_first_date = (
    df.groupby(COL_USER_ID)["event_ts"].min().dt.to_period("M").rename("cohort_month")
)

df = df.join(user_first_date, on=COL_USER_ID)

df["event_month"] = df["event_ts"].dt.to_period("M")


In [28]:
from operator import attrgetter   # only needed if using attrgetter

# STEP 1 — Convert timestamps to monthly periods
df['cohort_month'] = df[COL_EVENT_TIME].dt.to_period('M')
df['event_month']  = df[COL_EVENT_TIME].dt.to_period('M')

# STEP 2 — Active users per cohort x month
cohort_pivot = (
    df.groupby(['cohort_month', 'event_month'])[COL_USER_ID]
    .nunique()
    .rename('active_users')
    .reset_index()
)

# STEP 3 — Cohort size
cohort_sizes = (
    df.groupby('cohort_month')[COL_USER_ID]
    .nunique()
    .rename('cohort_size')
)

# STEP 4 — Merge
cohort_pivot = cohort_pivot.merge(
    cohort_sizes,
    on='cohort_month',
    how='left'
)

# STEP 5 — Month offset
from operator import attrgetter  # put this at the top of the notebook if not already

delta = cohort_pivot['event_month'] - cohort_pivot['cohort_month']
cohort_pivot['month_offset'] = delta.apply(attrgetter("n")).astype(int)


# STEP 6 — Retention rate
cohort_pivot['retention_rate'] = (
    cohort_pivot['active_users'] / cohort_pivot['cohort_size']
)

cohort_pivot.head()


,cohort_month,event_month,active_users,cohort_size,month_offset,retention_rate
0,2025-11,2025-11,8912,8912,0,1.0


In [29]:
retention_matrix = cohort_pivot.pivot_table(
    index="cohort_month",
    columns="month_offset",
    values="retention_rate"
)

retention_matrix


month_offset,0
cohort_month,
2025-11,1.0


In [34]:
# ---------- RFM + CLUSTERING (no order_id) ----------
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# assumes these are already defined:
# COL_EVENT_TIME  = "event_ts"
# COL_EVENT_NAME  = "event_type"
# COL_USER_ID     = "user_id"
# COL_REVENUE     = "price"

df[COL_EVENT_TIME] = pd.to_datetime(df[COL_EVENT_TIME], errors="coerce")

today = df[COL_EVENT_TIME].max() + pd.Timedelta(days=1)

purchase_mask = df[COL_EVENT_NAME] == "purchase"

# Monetary = total price for purchase events
user_monetary = (
    df[purchase_mask]
      .groupby(COL_USER_ID)[COL_REVENUE]
      .sum()
      .rename("monetary")
)

# Frequency = number of purchase events per user
user_frequency = (
    df[purchase_mask]
      .groupby(COL_USER_ID)[COL_EVENT_TIME]  # or "event_id"
      .count()
      .rename("frequency")
)

# Recency = days since last event
user_last_event = df.groupby(COL_USER_ID)[COL_EVENT_TIME].max()
user_recency = ((today - user_last_event).dt.days).rename("recency_days")

rfm = pd.concat([user_recency, user_frequency, user_monetary], axis=1).fillna(0)

features = ["recency_days", "frequency", "monetary"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(rfm[features])

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm["cluster"] = kmeans.fit_predict(X_scaled)

rfm.head()


/tmp/ipython-input-1222728161.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rfm = pd.concat([user_recency, user_frequency, user_monetary], axis=1).fillna(0)


,recency_days,frequency,monetary,cluster
user_id,,,,
user_1,1,0.0,0,0
user_10,1,0.0,0,0
user_100,1,0.0,0,0
user_1000,4,0.0,0,3
user_1001,2,0.0,0,2


In [35]:
features = ["recency_days", "frequency", "monetary"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(rfm[features])

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm["cluster"] = kmeans.fit_predict(X_scaled)

rfm.groupby("cluster")[features].mean()


,recency_days,frequency,monetary
cluster,,,
0,1.000000,0.0,0.0
1,3.000000,0.0,0.0
2,2.000000,0.0,0.0
3,4.288288,0.0,0.0


In [36]:
rfm.to_csv("user_segments_rfm.csv")


In [38]:
df = pd.read_csv("clean_events.csv", parse_dates=["event_ts"])

search_df = df[df[COL_EVENT_NAME] == "search"].copy()

search_df["event_date"] = search_df["event_ts"].dt.date

# Top keywords overall
top_keywords = (
    search_df.groupby(COL_SEARCH_KW)[COL_USER_ID]
    .nunique()
    .sort_values(ascending=False)
    .head(20)
)

top_keywords


,user_id
search_keyword,
sports,219
shoes,216
electronics,207
movies,201
news,198
books,196
games,193
fashion,186
music,178


In [39]:
# Keywords with no subsequent product_click or purchase in same session
# (simple approximation of "no good results")

# Users who searched
search_sessions = search_df["session_id"].unique()

# Sessions where any product_click or purchase happened
engaged_sessions = df[
    df["session_id"].isin(search_sessions)
    & df[COL_EVENT_NAME].isin(["product_click", "purchase"])
]["session_id"].unique()

search_df["had_result"] = search_df["session_id"].isin(engaged_sessions)

search_gap = (
    search_df.groupby(COL_SEARCH_KW)["had_result"]
    .mean()
    .rename("search_success_rate")
    .reset_index()
)

search_gap["gap_rate"] = 1 - search_gap["search_success_rate"]
search_gap.sort_values("gap_rate", ascending=False).head(20)


,search_keyword,search_success_rate,gap_rate
0,books,0.0,1.0
1,electronics,0.0,1.0
2,fashion,0.0,1.0
3,games,0.0,1.0
4,movies,0.0,1.0
5,music,0.0,1.0
6,news,0.0,1.0
7,shoes,0.0,1.0
8,sports,0.0,1.0


In [41]:
# read sessions (make sure session_id is a column, not index)
session_agg = pd.read_csv(
    "sessions.csv",
    parse_dates=["session_start", "session_end"]
)

# Attach network info: majority network per session
if COL_NETWORK and COL_NETWORK in df.columns:
    def most_common_network(x):
        vc = x.dropna().value_counts()
        return vc.index[0] if len(vc) > 0 else np.nan

    session_network = (
        df.groupby("session_id")[COL_NETWORK]
          .agg(most_common_network)
          .reset_index()
          .rename(columns={COL_NETWORK: "network_type"})
    )

    session_agg = session_agg.merge(session_network, on="session_id", how="left")

    session_agg.groupby("network_type")["session_quality_score"].mean()


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
import pandas as pd

# assumes:
# COL_EVENT_TIME  = "event_ts"
# COL_EVENT_NAME  = "event_type"
# COL_USER_ID     = "user_id"
# COL_REVENUE     = "price"

df = pd.read_csv("clean_events.csv", parse_dates=[COL_EVENT_TIME])

# Last activity date per user
user_last_event = df.groupby(COL_USER_ID)[COL_EVENT_TIME].max()

cutoff_date = df[COL_EVENT_TIME].max() - pd.Timedelta(days=30)

labels = (user_last_event < cutoff_date).astype(int)  # 1 = churned
labels.name = "churned"

# Simple features: total_events, total_purchases, total_revenue, avg_session_quality
user_total_events = df.groupby(COL_USER_ID).size().rename("total_events")

user_total_purchases = (
    df[df[COL_EVENT_NAME] == "purchase"]
      .groupby(COL_USER_ID)[COL_EVENT_TIME]      # count purchase events
      .count()
      .rename("total_purchases")
)

user_total_revenue = (
    df[df[COL_EVENT_NAME] == "purchase"]
      .groupby(COL_USER_ID)[COL_REVENUE]
      .sum()
      .rename("total_revenue")
)

session_agg = pd.read_csv("sessions.csv")

user_avg_session_quality = (
    session_agg.groupby("user_id")["session_quality_score"]
      .mean()
      .rename("avg_session_quality")
)

X = pd.concat(
        [user_total_events, user_total_purchases, user_total_revenue, user_avg_session_quality],
        axis=1,
    ).fillna(0)
X = X.infer_objects(copy=False)

y = labels.reindex(X.index)

# drop users with missing labels (if any)
mask = y.notna()
X = X[mask]
y = y[mask].astype(int)

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42,
    class_weight="balanced"
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# handle case where only one class is present
if len(model.classes_) > 1 and y_test.nunique() > 1:
    proba = model.predict_proba(X_test)
    y_pred_proba = proba[:, 1]
    print("AUC:", roc_auc_score(y_test, y_pred_proba))
else:
    print("AUC not defined (only one class in y).")

print(classification_report(y_test, y_pred))


/tmp/ipython-input-3432490010.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)


AUC not defined (only one class in y).
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2674

    accuracy                           1.00      2674
   macro avg       1.00      1.00      1.00      2674
weighted avg       1.00      1.00      1.00      2674



In [48]:
# train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42,
    class_weight="balanced"
)

model.fit(X_train, y_train)

# handle single-class case safely
if len(model.classes_) > 1 and y_test.nunique() > 1:
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    print("AUC:", roc_auc_score(y_test, y_pred_proba))
else:
    y_pred = model.predict(X_test)
    print("AUC not defined (only one class present in y).")

print(classification_report(y_test, y_pred))

feature_importance = (
    pd.Series(model.feature_importances_, index=X.columns)
      .sort_values(ascending=False)
)
feature_importance


AUC not defined (only one class present in y).
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2674

    accuracy                           1.00      2674
   macro avg       1.00      1.00      1.00      2674
weighted avg       1.00      1.00      1.00      2674



,0
total_events,0.0
total_purchases,0.0
total_revenue,0.0
avg_session_quality,0.0
